In [1]:
import pandas as pd
import numpy as np
import cvxpy

In [2]:
train = pd.read_csv('../data/train.csv')
sub = pd.read_csv('../data/sample_submission.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
file_configs = {
    '1JHC': [
        '../results/each_type/distance_kernel_1JHC.npy',
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_1JHC.npy',
        '../results/each_type/400_feats_types_split_models_1JHC.npy',
        '../results/each_type/giba_features_1JHC.npy',
        '../results/each_type/types_split_models_1JHC.npy',
        '../results/each_type/results_from_best_feats_1JHC.npy',
        '../results/each_type/knn_and_distance_kernel_1JHC.npy',
        '../results/each_type/knn_and_distance_kernel_1JHC_nfolds_100_topk_15.npy',
    ],
    '1JHN': [
        '../results/each_type/distance_kernel_1JHN.npy',
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_1JHN.npy',
        '../results/each_type/400_feats_types_split_models_1JHN.npy',
        '../results/each_type/groupby_distance_kernel_1JHN.npy',
        '../results/each_type/giba_features_1JHN.npy',
        '../results/each_type/types_split_models_1JHN.npy',
        '../results/each_type/results_from_best_feats_1JHN.npy',
        '../results/each_type/knn_and_distance_kernel_1JHN.npy',
        '../results/each_type/knn_and_distance_kernel_1JHN_nfolds_100_topk_15.npy',
    ],
    '2JHC': [
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_2JHC.npy',
        '../results/each_type/400_feats_types_split_models_2JHC.npy',
        '../results/each_type/distance_kernel_2JHC.npy',
        '../results/each_type/giba_features_2JHC.npy',
        '../results/each_type/results_from_best_feats_2JHC.npy',
        '../results/each_type/types_split_models_2JHC.npy',
        '../results/each_type/knn_and_distance_kernel_2JHC_nfolds_100_topk_15.npy',
    ],
    '2JHN': [
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_2JHN.npy',
        '../results/each_type/400_feats_types_split_models_2JHN.npy',
        '../results/each_type/distance_kernel_2JHN.npy',
        '../results/each_type/giba_features_2JHN.npy',
        '../results/each_type/groupby_distance_kernel_2JHN.npy',
        '../results/each_type/results_from_best_feats_2JHN.npy',
        '../results/each_type/tuned_kernel_distance_2JHN.npy',
        '../results/each_type/types_split_models_2JHN.npy',
#         '../results/each_type/knn_and_distance_kernel_2JHN.npy',
        '../results/each_type/knn_and_distance_kernel_2JHN_nfolds_100_topk_15.npy',
    ],
    '2JHH': [
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_2JHH.npy',
        '../results/each_type/400_feats_types_split_models_2JHH.npy',
        '../results/each_type/distance_kernel_2JHH.npy',
        '../results/each_type/giba_features_2JHH.npy',
        '../results/each_type/groupby_distance_kernel_2JHH.npy',
        '../results/each_type/results_from_best_feats_2JHH.npy',
        '../results/each_type/types_split_models_2JHH.npy',
        '../results/each_type/knn_and_distance_kernel_2JHH.npy',
        '../results/each_type/knn_and_distance_kernel_2JHH_nfolds_100_topk_15.npy',
    ],
    '3JHC': [
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_3JHC.npy',
        '../results/each_type/400_feats_types_split_models_3JHC.npy',
        '../results/each_type/distance_kernel_3JHC.npy',
        '../results/each_type/giba_features_3JHC.npy',
        '../results/each_type/results_from_best_feats_3JHC.npy',
        '../results/each_type/types_split_models_3JHC.npy',
        '../results/each_type/knn_and_distance_kernel_3JHC_nfolds_100_topk_15.npy',

    ],
    '3JHH': [
        '../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_3JHH.npy',
        '../results/each_type/400_feats_types_split_models_3JHH.npy',
        '../results/each_type/distance_kernel_3JHH.npy',
        '../results/each_type/giba_features_3JHH.npy',
        '../results/each_type/results_from_best_feats_3JHH.npy',
        '../results/each_type/types_split_models_3JHH.npy',
        '../results/each_type/tuned_kernel_distance_3JHH.npy',
        '../results/each_type/knn_and_distance_kernel_3JHH_nfolds_100_topk_15.npy',
    ],
    '3JHN': [
        '../results/each_type/400_feats_types_split_models_3JHN.npy',
        '../results/each_type/distance_kernel_3JHN.npy',
        '../results/each_type/giba_features_3JHN.npy',
        '../results/each_type/groupby_distance_kernel_3JHN.npy',
        '../results/each_type/results_from_best_feats_3JHN.npy',
        '../results/each_type/types_split_models_3JHN.npy',
        '../results/each_type/tuned_kernel_distance_3JHN.npy',
        '../results/each_type/knn_and_distance_kernel_3JHN_nfolds_100_topk_15.npy',
    ]
}

In [4]:
from sklearn.linear_model import LinearRegression, RidgeCV

In [5]:
scores = []
for type_mol in train.type.unique():
# for type_mol in ['3JHN']:
    print(f'type {type_mol}')
    train_mask = train.type == type_mol
    train_type = train[train_mask]
    
    files = file_configs[type_mol]
    number_of_files = len(files)
    
    res_oof = [np.load(file, allow_pickle=True).item()['oof'] for file in files]
    
    for i, cur_res_oof in enumerate(res_oof):
        cur_error = np.log(np.mean(np.abs(cur_res_oof - train_type['scalar_coupling_constant'])))
        print(f'file {files[i]}, error {cur_error}')
    
    res_preds = [np.load(file, allow_pickle=True).item()['prediction'] for file in files]
    
    X_train = np.vstack(res_oof).T
    X_test = np.vstack(res_preds).T
    
    y_train = train_type['scalar_coupling_constant']
    lr = RidgeCV(
        alphas=[10e-4, 10e-3, 10e-2, 0.05, 0.1, 0.5, 1., 5., 10., 100., 500, 1000, 10000],
        fit_intercept=False,
        cv=20,
    )

    lr.fit(X_train, y_train)
    oof_preds = lr.predict(X_train)
    
    
    error = np.mean(np.abs(oof_preds - train_type['scalar_coupling_constant']))
    error = np.log(error)
    print('*' * 40)
    print(f'type {type_mol}, error {error}')
    
    test_preds = lr.predict(X_test)
    test_mask = test.type == type_mol
    test.loc[test_mask, 'prediction'] = test_preds
    
    scores.append(error)

type 1JHC
file ../results/each_type/distance_kernel_1JHC.npy, error -0.49523276998748106
file ../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_1JHC.npy, error -0.5763950288110059
file ../results/each_type/400_feats_types_split_models_1JHC.npy, error -0.6174951310682736
file ../results/each_type/giba_features_1JHC.npy, error -0.2720270977098093
file ../results/each_type/types_split_models_1JHC.npy, error -0.5762878074316797
file ../results/each_type/results_from_best_feats_1JHC.npy, error -0.5314412672747475
file ../results/each_type/knn_and_distance_kernel_1JHC.npy, error -0.4536219478972443
file ../results/each_type/knn_and_distance_kernel_1JHC_nfolds_100_topk_15.npy, error -0.3749027033354864
****************************************
type 1JHC, error -0.7498670699670388
type 2JHH
file ../results/each_type/100k_iters_distance_feats_200_feats_types_split_models_2JHH.npy, error -1.9216928842184604
file ../results/each_type/400_feats_types_split_models_2JHH.npy,

In [6]:
np.mean(scores)

-1.8113975091506027

In [7]:
sub['scalar_coupling_constant'] = test['prediction']
sub.to_csv('../submissions/lin_regression_with_cv.csv', index=False)

In [8]:
lr.alpha_

10.0